GPT2 is in pytorch_transformers

In [ ]:
import sys
# You may need to play around with the path like I did to find packages
!{sys.executable} -m pip install numpy -qU
!{sys.executable} -m pip install scipy -qU
!{sys.executable} -m pip install pytorch_transformers -qU

Import required libraries

In [2]:
print(sys.path)
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']


Load pre-trained model tokenizer (vocabulary)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

100%|██████████| 456318/456318 [00:00<00:00, 1841407.07B/s]


Encode the sentence to complete

In [4]:
text = "What mammal has over 40,000 muscles in one body part?"
indexed_tokens = tokenizer.encode(text)

Convert indexed tokens in a PyTorch tensor

In [5]:
tokens_tensor = torch.tensor([indexed_tokens])

Load pre-trained model (weights)

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

100%|██████████| 548118077/548118077 [00:12<00:00, 42716034.21B/s]
/usr/local/lib/python3.11/dist-packages/pytorch_transformers/modeling_utils.py:539: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

Set the model in evaluation mode to deactivate the DropOut modules

In [7]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

If you have a GPU, uncomment the following to put everything on cuda

In [ ]:
# tokens_tensor = tokens_tensor.to('cuda')
# model.to('cuda')

Predict all tokens

In [8]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

Get the predicted next sub-word

In [9]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

Print the predicted word

In [10]:
print(predicted_text)

 What mammal has over 40,000 muscles in one body part?



Examples

In [11]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "the oldest university in Illinois is"
indexed_tokens = tokenizer.encode(text)

tokens_tensor = torch.tensor([indexed_tokens])

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()


with torch.no_grad():
    for token in range(10):  # Generate 10 additional tokens
        outputs = model(tokens_tensor)
        predictions = outputs[0]

        predicted_index = torch.argmax(predictions[0, -1, :]).item()

        tokens_tensor = torch.cat((tokens_tensor, torch.tensor([[predicted_index]])), dim=1)

predicted_text = tokenizer.decode(tokens_tensor[0].tolist(), skip_special_tokens=True)

print(predicted_text)


 the oldest university in Illinois is now a full-fledged university.

The


In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "the first president of the USA was President"
indexed_tokens = tokenizer.encode(text)

tokens_tensor = torch.tensor([indexed_tokens])

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()


with torch.no_grad():
    for token in range(3):  # Generate 10 additional tokens
        outputs = model(tokens_tensor)
        predictions = outputs[0]

        predicted_index = torch.argmax(predictions[0, -1, :]).item()

        tokens_tensor = torch.cat((tokens_tensor, torch.tensor([[predicted_index]])), dim=1)

predicted_text = tokenizer.decode(tokens_tensor[0].tolist(), skip_special_tokens=True)

print(predicted_text)



 the first president of the USA was President George W.


In [13]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "the first president of the USA was President"
indexed_tokens = tokenizer.encode(text)

tokens_tensor = torch.tensor([indexed_tokens])

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()


with torch.no_grad():
    for token in range(10):  # Generate 10 additional tokens
        outputs = model(tokens_tensor)
        predictions = outputs[0]

        predicted_index = torch.argmax(predictions[0, -1, :]).item()

        tokens_tensor = torch.cat((tokens_tensor, torch.tensor([[predicted_index]])), dim=1)

predicted_text = tokenizer.decode(tokens_tensor[0].tolist(), skip_special_tokens=True)

print(predicted_text)



 the first president of the USA was President George W. Bush.

The first president


By generating more tokens we can create longer statements.
However they seem to not really make sense or seem coherent
since each token is chosen statistically and the best fit from a distribution of next tokens. This is seen in the last prompt where 10 tokens are generated for the name of the first USA president.

Also the prompts have to be very specific and crafted in a way that leaves no room for ambiguity


In [18]:
#  Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# # Navigate to your working directory
%cd /content/gdrive/MyDrive/fall_2024_uchicago/generative_ai/wk_1

Mounted at /content/gdrive


In [26]:
!touch /content/gdrive/MyDrive/fall_2024_uchicago/generative_ai/wk_1/gpt2_script.py

In [29]:
%%writefile /content/gdrive/MyDrive/fall_2024_uchicago/generative_ai/wk_1/gpt2_script.py

import torch
import sys
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 50  # Maximum length of generated text
MODEL_NAME = 'gpt2'

def generate_text(prompt, max_new_tokens=10):
    """
    Generate text continuation from a prompt using GPT-2

    Args:
        prompt (str): Input text to continue
        max_new_tokens (int): Number of new tokens to generate

    Returns:
        str: Generated text including prompt
    """
    # Encode prompt
    indexed_tokens = tokenizer.encode(prompt)
    tokens_tensor = torch.tensor([indexed_tokens]).to(DEVICE)

    # Generate tokens
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(tokens_tensor)
            predictions = outputs[0]
            predicted_index = torch.argmax(predictions[0, -1, :]).item()
            tokens_tensor = torch.cat((tokens_tensor,
                                    torch.tensor([[predicted_index]]).to(DEVICE)),
                                    dim=1)

    # Decode and return text
    return tokenizer.decode(tokens_tensor[0].tolist(), skip_special_tokens=True)

# 4. Add example usage function
def run_examples():
    """Run example text generations with different prompts"""
    examples = [
        "What mammal has over 40,000 muscles in one body part?",
        "the oldest university in Illinois is",
        "the first president of the USA was President"
    ]

    for prompt in examples:
        print(f"\nPrompt: {prompt}")
        print(f"Generated: {generate_text(prompt)}")
        print("-" * 50)

run_examples()

Overwriting /content/gdrive/MyDrive/fall_2024_uchicago/generative_ai/wk_1/gpt2_script.py
